
<br>
Exploration d'une jeu de données de réservations d'hôtel issu du site Kaggle.<br>


In [1]:
from collections import Counter
import calendar

Je lis le contenu du csv.

In [2]:
csvpath = "./hotel_reservations.csv"
with open(csvpath, "r") as f:
    csv = f.readlines()

Je transforme chaque ligne en une liste. L'ensemble est donc une liste de listes.

In [3]:
csv = [line.split(",") for line in csv]
csv[:1]

[['Booking_ID',
  'no_of_adults',
  'no_of_children',
  'no_of_weekend_nights',
  'no_of_week_nights',
  'type_of_meal_plan',
  'required_car_parking_space',
  'room_type_reserved',
  'lead_time',
  'arrival_year',
  'arrival_month',
  'arrival_date',
  'market_segment_type',
  'repeated_guest',
  'no_of_previous_cancellations',
  'no_of_previous_bookings_not_canceled',
  'avg_price_per_room',
  'no_of_special_requests',
  'booking_status\n']]

Pour pouvoir plus facilement travailler, je transforme la liste de listes en dictionnaires de dictionnaires (nested dicts). L'ensemble est un dictionnaire, et chaque ligne du csv (= chaque réservation) va constituer un sous-dictionnaire, dont les clés correspondront aux entêtes du csv (les intitulés des colonnes)

In [4]:
c = {}
for line in csv[1:]:
    no_reservation = line[0]
    c[no_reservation] = {}
    colonnes = [i for i in range(1, len(line))]
    for no in colonnes:
        c[no_reservation][csv[0][no]] = line[no]

Combien d'années différentes sont concernées par ces réservations?

In [5]:
years = [c[i]["arrival_year"] for i in c.keys()]
Counter(years)

Counter({'2018': 29761, '2017': 6514})

Y a-t-il des réservations pour tous les mois de l'années?

In [6]:
months = [c[i]["arrival_month"] for i in c.keys()]
Counter(months)

Counter({'10': 5317,
         '9': 4611,
         '8': 3813,
         '6': 3203,
         '12': 3021,
         '11': 2980,
         '7': 2920,
         '4': 2736,
         '5': 2598,
         '3': 2358,
         '2': 1704,
         '1': 1014})

Je crée quelques fonctions simples pour explorer quelques aspects de ces données.

Fonction qui retourne une liste de tuples: l'id de la réservation et la valeur d'une colonne à choix, entrée comme paramètre.

In [7]:
def query_id_col(col: str):
    a = [(i, c[i][col]) for i in c.keys()]
    return a

Fonction qui retourne un dictionnaire associant (clé) l'id de la réservation à (valeur) la valeur attribuée à la colonne choisie pour cette réservation.

In [8]:
def query_id_dict(col: str):
    di = {}
    for i in c.keys():
        di[i] = c[i][col]
    return di

Fonction qui retourne pour une colonne, un dictionnaire qui associe (clé) les valeurs existantes pour cette colonnes au (valeur) nombre de réservation avec cette valeur dans cette colonne.

In [9]:
def query_valeur_nb(col: str):
    a = [(i, c[i][col]) for i in c.keys()]
    values = {}
    for i in a:
        if i[1] not in values.keys():
            values[i[1]] = 0
        else:
            values[i[1]] = values[i[1]] + 1
    return values

Fonction qui retourne, pour une colonne, un dictionnaire associant (clé) les valeurs existantes pour cette colonne à (valeur) une liste des ids des réservations ayant cette valeur.

In [10]:
def query_valeur_id(col: str):
    a = [(i, c[i][col]) for i in c.keys()]
    values = {}
    for i in a:
        if i[1] not in values.keys():
            values[i[1]] = []
        else:
            values[i[1]].append(i[0])
    return values

Question qui nous permettra peut-etre d'en apprendre davantage sur le type d'établissement: les saisons de réservations.

In [11]:
months = query_valeur_nb("arrival_month")
for m in months.keys():
    j = "|" * int(months[m] / 100)
    print(calendar.month_name[int(m)][:3], j, months[m])

Oct ||||||||||||||||||||||||||||||||||||||||||||||||||||| 5316
Nov ||||||||||||||||||||||||||||| 2979
Feb ||||||||||||||||| 1703
May ||||||||||||||||||||||||| 2597
Apr ||||||||||||||||||||||||||| 2735
Sep |||||||||||||||||||||||||||||||||||||||||||||| 4610
Dec |||||||||||||||||||||||||||||| 3020
Jul ||||||||||||||||||||||||||||| 2919
Jun |||||||||||||||||||||||||||||||| 3202
Aug |||||||||||||||||||||||||||||||||||||| 3812
Mar ||||||||||||||||||||||| 2357
Jan |||||||||| 1013


Pour y voir un peu mieux, mettre les mois dans l'ordre, et répéter l'opération.

In [12]:
months_sorted = [(int(m), months[m]) for m in months.keys()]
months_sorted.sort()
months_sorted
for m in months_sorted:
    j = "|" * int(m[1] / 100)
    print(calendar.month_name[m[0]][:3], j, m[1])

Jan |||||||||| 1013
Feb ||||||||||||||||| 1703
Mar ||||||||||||||||||||||| 2357
Apr ||||||||||||||||||||||||||| 2735
May ||||||||||||||||||||||||| 2597
Jun |||||||||||||||||||||||||||||||| 3202
Jul ||||||||||||||||||||||||||||| 2919
Aug |||||||||||||||||||||||||||||||||||||| 3812
Sep |||||||||||||||||||||||||||||||||||||||||||||| 4610
Oct ||||||||||||||||||||||||||||||||||||||||||||||||||||| 5316
Nov ||||||||||||||||||||||||||||| 2979
Dec |||||||||||||||||||||||||||||| 3020


Vraisemblablement, il ne s'agit pas d'un hôtel dont le public-cible est constitué de skieureuses. Les mois d'hivers et du début du printemps sont ceux pour lesquels il y a le moins de réservations.

La colonne "required_car_parking_space".

In [13]:
query_valeur_nb("required_car_parking_space")

{'0': 35150, '1': 1123}

Le champ "required_car_parking_space" me semble être intéressant à croiser avec d'autres champs. Par exemple, y a-t-il un rapport entre le mois de la réservation et le fait d'avoir besoin d'une place de parking? Peut-être qu'en été les gens viennent à pieds dans cet hôtel.

Pour chaque mois, le nombre de réservation avec une place de parking, et le nombre de réservation sans place de parking.<br>
months_id = query_id_dict("required_car_parking_space")<br>
parking = query_valeur_id("required_car_parking_space")

In [14]:
months = query_valeur_id("arrival_month")
parking = query_id_dict("required_car_parking_space")
a = {}
for i in months.keys():
    a[i] = {}
    a[i]["0"] = 0
    a[i]["1"] = 0
    for r in months[i]:
        if parking[r] == "0":
            a[i]["0"] = a[i]["0"] + 1
        elif parking[r] == "1":
            a[i]["1"] = a[i]["1"] + 1
        else:
            pass
    a[i]["total"] = a[i]["0"] + a[i]["1"]
    a[i]["proportion"] = round(a[i]["1"] / a[i]["total"], 3)

On peut voir des différences importantes dans les proportions de réservations avec voitures: de 0.018 par réservation en octobre à 0.058 en Aout.

In [15]:
for i in a.keys():
    print(calendar.month_name[int(i)][:3], a[i])

Oct {'0': 5222, '1': 94, 'total': 5316, 'proportion': 0.018}
Nov {'0': 2897, '1': 82, 'total': 2979, 'proportion': 0.028}
Feb {'0': 1650, '1': 53, 'total': 1703, 'proportion': 0.031}
May {'0': 2532, '1': 65, 'total': 2597, 'proportion': 0.025}
Apr {'0': 2670, '1': 65, 'total': 2735, 'proportion': 0.024}
Sep {'0': 4509, '1': 101, 'total': 4610, 'proportion': 0.022}
Dec {'0': 2918, '1': 102, 'total': 3020, 'proportion': 0.034}
Jul {'0': 2801, '1': 118, 'total': 2919, 'proportion': 0.04}
Jun {'0': 3129, '1': 73, 'total': 3202, 'proportion': 0.023}
Aug {'0': 3590, '1': 222, 'total': 3812, 'proportion': 0.058}
Mar {'0': 2262, '1': 95, 'total': 2357, 'proportion': 0.04}
Jan {'0': 959, '1': 54, 'total': 1013, 'proportion': 0.053}


Calculer l'écart.

In [16]:
proportions = [a[i]["proportion"] for i in a.keys()]
proportions.sort()
print(
    proportions[0],
    "/",
    proportions[-1],
    "=",
    round(proportions[-1] / proportions[0], 1),
)

0.018 / 0.058 = 3.2


Print les mois par la valeur "proportion".

In [17]:
p = [
    (a[i]["proportion"], calendar.month_name[int(i)][:3])
    for i in a.keys()
]
p.sort()
p.reverse()

In [18]:
for n, m in p:
    print(m, ":", n)

Aug : 0.058
Jan : 0.053
Mar : 0.04
Jul : 0.04
Dec : 0.034
Feb : 0.031
Nov : 0.028
May : 0.025
Apr : 0.024
Jun : 0.023
Sep : 0.022
Oct : 0.018


Une hypothèse que je formule à partir de ces résultats: les réservations de place de parking sont corrélées avec le nombre d'enfants. Des vacances d'été (s'il y en a dans le pays dans lequel se trouve cet hôtel, où dans les pays dans lesquels vivent ses clients) dans une période (disons) standardisée pour l'ensemble d'une population pourrait être une explication pour ces disparités. Je commence par faire la même opération mais avec la colonne "no_of_children".

In [19]:
months = query_valeur_id("arrival_month")
children = query_id_dict("no_of_children")
b = {}
for i in months.keys():
    b[i] = {}
    b[i]["0"] = 0
    b[i]["1"] = 0
    for r in months[i]:
        if children[r] == "0":
            b[i]["0"] = b[i]["0"] + 1
        elif int(children[r]) > 0:
            b[i]["1"] = b[i]["1"] + 1
        else:
            pass
    b[i]["total"] = b[i]["0"] + b[i]["1"]
    b[i]["proportion"] = round(b[i]["1"] / b[i]["total"], 3)

On peut voir des différences importantes dans les proportions de réservations avec voitures: de 0.018 par réservation en octobre à 0.058 en Aout.

In [20]:
for i in b.keys():
    print(calendar.month_name[int(i)][:3], b[i])

Oct {'0': 5035, '1': 281, 'total': 5316, 'proportion': 0.053}
Nov {'0': 2861, '1': 118, 'total': 2979, 'proportion': 0.04}
Feb {'0': 1583, '1': 120, 'total': 1703, 'proportion': 0.07}
May {'0': 2438, '1': 159, 'total': 2597, 'proportion': 0.061}
Apr {'0': 2521, '1': 214, 'total': 2735, 'proportion': 0.078}
Sep {'0': 4387, '1': 223, 'total': 4610, 'proportion': 0.048}
Dec {'0': 2723, '1': 297, 'total': 3020, 'proportion': 0.098}
Jul {'0': 2538, '1': 381, 'total': 2919, 'proportion': 0.131}
Jun {'0': 3048, '1': 154, 'total': 3202, 'proportion': 0.048}
Aug {'0': 3283, '1': 529, 'total': 3812, 'proportion': 0.139}
Mar {'0': 2177, '1': 180, 'total': 2357, 'proportion': 0.076}
Jan {'0': 971, '1': 42, 'total': 1013, 'proportion': 0.041}


Calculer l'écart

In [21]:
proportions = [b[i]["proportion"] for i in b.keys()]
proportions.sort()
print(
    proportions[0],
    "/",
    proportions[-1],
    "=",
    round(proportions[-1] / proportions[0], 1),
)

0.04 / 0.139 = 3.5


In [22]:
q = [
    (b[i]["proportion"], calendar.month_name[int(i)][:3])
    for i in b.keys()
]
q.sort()
q.reverse()

In [23]:
for n, m in q:
    print(m, ":", n)

Aug : 0.139
Jul : 0.131
Dec : 0.098
Apr : 0.078
Mar : 0.076
Feb : 0.07
May : 0.061
Oct : 0.053
Sep : 0.048
Jun : 0.048
Jan : 0.041
Nov : 0.04


Maintenant, essayer de voir si les familles avec enfant sont aussi les familles avec parking.

In [24]:
months = query_valeur_id("arrival_month")
parking = query_id_dict("required_car_parking_space")
children = query_id_dict("no_of_children")
c = {}
for i in months.keys():
    c[i] = {}
    c[i]["np_nc"] = 0
    c[i]["p_c"] = 0
    c[i]["np_c"] = 0
    c[i]["p_nc"] = 0
    for r in months[i]:
        if parking[r] == "0":
            if children[r] == "0":
                c[i]["np_nc"] = c[i]["np_nc"] + 1
            elif int(children[r]) > 0:
                c[i]["np_c"] = c[i]["np_c"] + 1
        elif parking[r] != "0":
            if children[r] == "0":
                c[i]["p_nc"] = c[i]["p_nc"] + 1
            elif int(children[r]) > 0:
                c[i]["p_c"] = c[i]["p_c"] + 1
    # a[i]["total"] = a[i]["0"] + a[i]["1"]
    # a[i]["proportion"] = round(a[i]["1"] / a[i]["total"], 3)

In [25]:
for i in c.keys():
    print(calendar.month_name[int(i)][:3], c[i])

Oct {'np_nc': 4948, 'p_c': 7, 'np_c': 274, 'p_nc': 87}
Nov {'np_nc': 2782, 'p_c': 3, 'np_c': 115, 'p_nc': 79}
Feb {'np_nc': 1535, 'p_c': 5, 'np_c': 115, 'p_nc': 48}
May {'np_nc': 2375, 'p_c': 2, 'np_c': 157, 'p_nc': 63}
Apr {'np_nc': 2458, 'p_c': 2, 'np_c': 212, 'p_nc': 63}
Sep {'np_nc': 4293, 'p_c': 7, 'np_c': 216, 'p_nc': 94}
Dec {'np_nc': 2629, 'p_c': 8, 'np_c': 289, 'p_nc': 94}
Jul {'np_nc': 2446, 'p_c': 26, 'np_c': 355, 'p_nc': 92}
Jun {'np_nc': 2985, 'p_c': 10, 'np_c': 144, 'p_nc': 63}
Aug {'np_nc': 3116, 'p_c': 55, 'np_c': 474, 'p_nc': 167}
Mar {'np_nc': 2088, 'p_c': 6, 'np_c': 174, 'p_nc': 89}
Jan {'np_nc': 923, 'p_c': 6, 'np_c': 36, 'p_nc': 48}
